## Convert data

https://doi.org/10.1016/j.isci.2022.104097

High Resolution Slide-seqV2 Spatial Transcriptomics Enables Discovery of Disease-Specific Cell Neighborhoods and Pathways

Spatial transcriptomics in the healthy human kidney: Puck_200903_13

Need to download from https://cellxgene.cziscience.com/collections/8e880741-bf9a-4c8e-9227-934204631d2a

Download to `./data/marshall_2022_iscience.h5ad`

In [ ]:
import scanpy as sc
import numpy as np
import scipy.cluster
from scipy import sparse

from anndata import read_h5ad
from os.path import join

from vitessce import (
    VitessceConfig,
    ViewType as vt,
    CoordinationType as ct,
    FileType as ft,
    AnnDataWrapper,
    OmeTiffWrapper,
)
from vitessce.data_utils import (
    to_diamond,
    to_uint8,
    optimize_adata,
)

In [ ]:
slide_seq_h5ad = join("data", "marshall_2022_iscience.h5ad")
slide_seq_zarr = join("data", "marshall_2022_iscience.h5ad.zarr")

In [ ]:
adata = read_h5ad(slide_seq_h5ad)

sc.pp.filter_cells(adata, min_genes=200)
sc.pp.filter_genes(adata, min_cells=3)

adata.var['mt'] = adata.var['feature_name'].str.startswith('MT-')  # annotate the group of mitochondrial genes as 'mt'
sc.pp.calculate_qc_metrics(adata, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)

sc.pp.normalize_total(adata, target_sum=1e4, inplace=True)
sc.pp.log1p(adata)

sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5)

adata_hvg = adata[:, adata.var['highly_variable']].copy()
sc.pp.regress_out(adata_hvg, ['total_counts', 'pct_counts_mt'])
sc.pp.scale(adata_hvg, max_value=3)

adata.obsm['X_hvg'] = adata_hvg.X
adata.obsm['X_hvg_uint8'] = to_uint8(adata_hvg.X, norm_along="var")

num_cells = adata.obs.shape[0]
adata.obsm['X_spatial'] = adata.obsm['X_spatial']
adata.obsm['X_segmentations'] = np.zeros((num_cells, 4, 2))
radius = 10
for i in range(num_cells):
    adata.obsm['X_segmentations'][i, :, :] = to_diamond(adata.obsm['X_spatial'][i, 0], adata.obsm['X_spatial'][i, 1], radius)

adata = optimize_adata(
    adata,
    obs_cols=["cell_type"],
    var_cols=["feature_name", "highly_variable"],
    obsm_keys=["X_hvg", "X_hvg_uint8", "X_umap", "X_spatial", "X_segmentations"],
    layer_keys=[],
)

adata.write_zarr(slide_seq_zarr, chunks=[adata.shape[0], 10])

## Configure visualization

In [ ]:
vc = VitessceConfig(schema_version="1.0.15", name='Slide-seqV2', description='Kidney')

In [ ]:
dataset = vc.add_dataset(name='Puck_200903_13').add_object(AnnDataWrapper(
    adata_path=slide_seq_zarr,
    obs_embedding_paths=["obsm/X_umap"],
    obs_embedding_names=["UMAP"],
    obs_locations_path="obsm/X_spatial",
    obs_segmentations_path="obsm/X_segmentations",
    obs_set_paths=["obs/cell_type"],
    obs_set_names=["Bead type"],
    # feature_labels_path="var/feature_name",
    obs_feature_matrix_path="obsm/X_hvg",
    feature_filter_path="var/highly_variable",
    coordination_values={
        "obsType": "bead"
    }
))

In [ ]:
obs_sets = vc.add_view(vt.OBS_SETS, dataset=dataset)
obs_set_sizes = vc.add_view(vt.OBS_SET_SIZES, dataset=dataset)
scatterplot = vc.add_view(vt.SCATTERPLOT, dataset=dataset, mapping="UMAP")
spatial = vc.add_view(vt.SPATIAL, dataset=dataset)
layer_controller = vc.add_view(vt.LAYER_CONTROLLER, dataset=dataset)
gene_list = vc.add_view(vt.FEATURE_LIST, dataset=dataset)

In [ ]:
vc.link_views([obs_sets, obs_set_sizes, scatterplot, spatial, layer_controller, gene_list], [ct.OBS_TYPE], ["bead"])

In [ ]:
vc.link_views([spatial, layer_controller], [ct.SPATIAL_SEGMENTATION_LAYER], [{ "opacity": 1, "radius": 50, "visible": True, "stroked": False }])

In [ ]:
vc.layout(
    (scatterplot | (gene_list | obs_sets))
    / (spatial | (layer_controller | obs_set_sizes))
);

In [ ]:
vc.to_dict(base_url="")

In [ ]:
vw = vc.widget()
vw